# Análisis Dimensional - Métodos VFLUX2

Este notebook realiza un análisis dimensional riguroso de cada método VFLUX2 para identificar errores de magnitud.

In [ ]:
import numpy as np
import pandas as pd

print("=" * 80)
print("ANÁLISIS DIMENSIONAL - MÉTODOS VFLUX2")
print("=" * 80)

ANÁLISIS DIMENSIONAL - MÉTODOS VFLUX2


## 1. Definir Parámetros Físicos y sus Unidades

In [2]:
# Parámetros térmicos
params = {
    'λ (lambda)': {'valor': 2.0, 'unidad': 'W/(m·K)', 'descripcion': 'Conductividad térmica'},
    'Cs': {'valor': 2.5e6, 'unidad': 'J/(m³·K)', 'descripcion': 'Capacidad calorífica sedimento'},
    'Cw': {'valor': 4.18e6, 'unidad': 'J/(m³·K)', 'descripcion': 'Capacidad calorífica agua'},
    'ω (omega)': {'valor': 2*np.pi/86400, 'unidad': 'rad/s', 'descripcion': 'Frecuencia angular diaria'},
    'α (alpha)': {'valor': 2.0/2.5e6, 'unidad': 'm²/s', 'descripcion': 'Difusividad térmica (λ/Cs)'},
}

# Mostrar tabla
print("\nParámetros Térmicos del Sedimento:")
print("-" * 80)
for nombre, info in params.items():
    print(f"{nombre:<15} = {info['valor']:<15.6e} {info['unidad']:<15} | {info['descripcion']}")

# Verificar α
alpha_calc = params['λ (lambda)']['valor'] / params['Cs']['valor']
print(f"\n✓ Verificación: α = λ/Cs = {alpha_calc:.6e} m²/s")


Parámetros Térmicos del Sedimento:
--------------------------------------------------------------------------------
λ (lambda)      = 2.000000e+00    W/(m·K)         | Conductividad térmica
Cs              = 2.500000e+06    J/(m³·K)        | Capacidad calorífica sedimento
Cw              = 4.180000e+06    J/(m³·K)        | Capacidad calorífica agua
ω (omega)       = 7.272205e-05    rad/s           | Frecuencia angular diaria
α (alpha)       = 8.000000e-07    m²/s            | Difusividad térmica (λ/Cs)

✓ Verificación: α = λ/Cs = 8.000000e-07 m²/s


## 2. Análisis Dimensional - Método Hatch-Phase

### Ecuación Teórica (Hatch et al., 2006):

Para flujo vertical en medios porosos con señal térmica sinusoidal:

$$v = \frac{4 \alpha \Delta\phi}{\omega \Delta z^2}$$

Donde:
- v: velocidad de Darcy [m/s]
- α: difusividad térmica [m²/s]
- Δφ: desfase de fase [rad]
- ω: frecuencia angular [rad/s]
- Δz: diferencia de profundidad [m]

In [ ]:
print("\n" + "=" * 80)
print("ANÁLISIS DIMENSIONAL: MÉTODO HATCH-PHASE")
print("=" * 80)

# Valores de entrada
alpha = params['α (alpha)']['valor']  # 8e-7 m²/s
omega = params['ω (omega)']['valor']  # 7.27e-5 rad/s
delta_phi = 0.4828  # rad (medido)
delta_z = 0.10  # m

print(f"\nValores de entrada:")
print(f"  α = {alpha:.6e} m²/s")
print(f"  ω = {omega:.6e} rad/s")
print(f"  Δφ = {delta_phi:.4f} rad")
print(f"  Δz = {delta_z:.2f} m")

# Análisis dimensional paso a paso
print(f"\nAnálisis dimensional:")
print(f"\n  v = (4 × α × Δφ) / (ω × Δz²)")

numerador = 4 * alpha * delta_phi
denominador = omega * delta_z**2

print(f"\n  Numerador = 4 × {alpha:.2e} × {delta_phi:.4f}")
print(f"            = {numerador:.6e}")
print(f"            Unidades: [adimensional] × [m²/s] × [rad] = [m²·rad/s]")

print(f"\n  Denominador = {omega:.2e} × {delta_z}²")
print(f"              = {denominador:.6e}")
print(f"              Unidades: [rad/s] × [m²] = [m²·rad/s]")

v_ms = numerador / denominador
print(f"\n  v = {numerador:.6e} / {denominador:.6e}")
print(f"    = {v_ms:.6e} m/s")
print(f"    Unidades: [m²·rad/s] / [m²·rad/s] = [adimensional] X ")

print(f"\nPROBLEMA: Los radianes se cancelan, resultado es adimensional, NO [m/s]!")


ANÁLISIS DIMENSIONAL: MÉTODO HATCH-PHASE

Valores de entrada:
  α = 8.000000e-07 m²/s
  ω = 7.272205e-05 rad/s
  Δφ = 0.4828 rad
  Δz = 0.10 m

📐 Análisis dimensional:

  v = (4 × α × Δφ) / (ω × Δz²)

  Numerador = 4 × 8.00e-07 × 0.4828
            = 1.544960e-06
            Unidades: [adimensional] × [m²/s] × [rad] = [m²·rad/s]

  Denominador = 7.27e-05 × 0.1²
              = 7.272205e-07
              Unidades: [rad/s] × [m²] = [m²·rad/s]

  v = 1.544960e-06 / 7.272205e-07
    = 2.124473e+00 m/s
    Unidades: [m²·rad/s] / [m²·rad/s] = [adimensional] ❌

⚠️  PROBLEMA: Los radianes se cancelan, resultado es adimensional, NO [m/s]!


## 3. Investigar Formulación Correcta

El problema es que los **radianes son adimensionales**, por lo que se cancelan en numerador y denominador.

Revisemos la ecuación original de Stallman/Bredehoeft para transporte de calor en medios porosos.

In [ ]:
print("\n" + "=" * 80)
print("TEORÍA: Ecuación de Transporte de Calor en Medios Porosos")
print("=" * 80)

print("""
Ecuación de Stallman (1965) para propagación de onda térmica con advección:

Para señal sinusoidal T(z,t) = A(z) × sin(ωt - φ(z)):

1. Atenuación de amplitud:
   A(z) = A₀ × exp(-z × β)
   
   donde β = √(ω/(2α)) × √(1 + Pe² + Pe)
   y Pe = v×L/(2α) = número de Péclet

2. Desfase de fase:
   φ(z) = z × √(ω/(2α)) × √(1 + Pe² - Pe)

Para flujos pequeños (Pe << 1), aproximación lineal:
   
   φ(z) ≈ z × √(ω/(2α)) + z × (v×Cw)/(2×λ)
        └─── término conductivo ───┘   └── término advectivo ──┘

Despejando v desde Δφ = φ(z₂) - φ(z₁):

   v ≈ (Δφ/Δz - √(ω/(2α))) × (2λ)/(Cw)

""")

print("\n HIPÓTESIS: La ecuación implementada NO incluye el término conductivo")


TEORÍA: Ecuación de Transporte de Calor en Medios Porosos

Ecuación de Stallman (1965) para propagación de onda térmica con advección:

Para señal sinusoidal T(z,t) = A(z) × sin(ωt - φ(z)):

1. Atenuación de amplitud:
   A(z) = A₀ × exp(-z × β)

   donde β = √(ω/(2α)) × √(1 + Pe² + Pe)
   y Pe = v×L/(2α) = número de Péclet

2. Desfase de fase:
   φ(z) = z × √(ω/(2α)) × √(1 + Pe² - Pe)

Para flujos pequeños (Pe << 1), aproximación lineal:

   φ(z) ≈ z × √(ω/(2α)) + z × (v×Cw)/(2×λ)
        └─── término conductivo ───┘   └── término advectivo ──┘

Despejando v desde Δφ = φ(z₂) - φ(z₁):

   v ≈ (Δφ/Δz - √(ω/(2α))) × (2λ)/(Cw)



⚡ HIPÓTESIS: La ecuación implementada NO incluye el término conductivo


## 4. Probar Ecuación Corregida

In [ ]:
print("\n" + "=" * 80)
print("PRUEBA: Ecuación Hatch-Phase Corregida")
print("=" * 80)

# Parámetros
lambda_val = 2.0  # W/(m·K)
Cw = 4.18e6  # J/(m³·K)
alpha = 8e-7  # m²/s
omega = 7.27e-5  # rad/s
delta_phi = 0.4828  # rad
delta_z = 0.10  # m

# Término conductivo puro
phi_conductivo_por_metro = np.sqrt(omega / (2 * alpha))
delta_phi_conductivo = phi_conductivo_por_metro * delta_z

print(f"\nTérmino conductivo:")
print(f"  φ_cond/Δz = √(ω/(2α)) = {phi_conductivo_por_metro:.4f} rad/m")
print(f"  Δφ_cond = {delta_phi_conductivo:.4f} rad (para Δz = {delta_z} m)")

# Desfase adicional por advección
delta_phi_advectivo = delta_phi - delta_phi_conductivo
print(f"\nDesfase medido total: {delta_phi:.4f} rad")
print(f"Desfase por conducción: {delta_phi_conductivo:.4f} rad")
print(f"Desfase por advección: {delta_phi_advectivo:.4f} rad")

# Ecuación corregida
if delta_phi_advectivo > 0:
    v_corregido = delta_phi_advectivo / delta_z * (2 * lambda_val) / Cw
    v_mm_day = v_corregido * 86400 * 1000
    
    print(f"\n✓ Flujo calculado (ecuación corregida):")
    print(f"  v = (Δφ_adv/Δz) × (2λ/Cw)")
    print(f"  v = ({delta_phi_advectivo:.4f}/{delta_z}) × (2×{lambda_val}/{Cw:.2e})")
    print(f"  v = {v_corregido:.6e} m/s")
    print(f"  v = {v_mm_day:.2f} mm/día")
    print(f"\n  Flujo objetivo: 5.00 mm/día")
    print(f"  Error: {abs(v_mm_day - 5.0):.2f} mm/día")
else:
    print(f"\n  Δφ_advectivo es negativo! El desfase medido es menor que el conductivo puro.")
    print(f"     Esto indica flujo ascendente o error en mediciones.")


PRUEBA: Ecuación Hatch-Phase Corregida

Término conductivo:
  φ_cond/Δz = √(ω/(2α)) = 6.7407 rad/m
  Δφ_cond = 0.6741 rad (para Δz = 0.1 m)

Desfase medido total: 0.4828 rad
Desfase por conducción: 0.6741 rad
Desfase por advección: -0.1913 rad

⚠️  Δφ_advectivo es negativo! El desfase medido es menor que el conductivo puro.
     Esto indica flujo ascendente o error en mediciones.


## 5. Verificar con Datos Sintéticos

Vamos a verificar cómo se generó el desfase en `generate_synthetic_data.py`

In [6]:
print("\n" + "=" * 80)
print("VERIFICACIÓN: Generación de Desfases Sintéticos")
print("=" * 80)

# Parámetros usados en generate_synthetic_data.py
TARGET_FLUX = 5.0  # mm/día
v_objetivo = TARGET_FLUX * 1e-3 / 86400  # m/s

print(f"\nFlujo objetivo: {TARGET_FLUX} mm/día = {v_objetivo:.6e} m/s")

# Fórmula usada en generate_synthetic_data.py (aproximación McCallum)
term_conductive = np.sqrt((omega * delta_z**2) / (4 * alpha))
term_advective = (v_objetivo * Cw * delta_z) / (2 * lambda_val)
delta_phi_generado = term_conductive + term_advective

print(f"\nFórmula de generación (McCallum aproximado):")
print(f"  Término conductivo: √((ω×Δz²)/(4α)) = {term_conductive:.4f} rad")
print(f"  Término advectivo: (v×Cw×Δz)/(2λ) = {term_advective:.6f} rad")
print(f"  Δφ total generado: {delta_phi_generado:.4f} rad")

print(f"\n  Δφ medido en notebook: {delta_phi:.4f} rad")
print(f"  Diferencia: {abs(delta_phi - delta_phi_generado):.6f} rad")

if abs(delta_phi - delta_phi_generado) < 0.001:
    print("\n  ✓ Coincide! El desfase medido corresponde al generado.")
else:
    print("\n  ✗ NO coincide. Hay discrepancia en la generación/medición.")


VERIFICACIÓN: Generación de Desfases Sintéticos

Flujo objetivo: 5.0 mm/día = 5.787037e-08 m/s

Fórmula de generación (McCallum aproximado):
  Término conductivo: √((ω×Δz²)/(4α)) = 0.4766 rad
  Término advectivo: (v×Cw×Δz)/(2λ) = 0.006047 rad
  Δφ total generado: 0.4827 rad

  Δφ medido en notebook: 0.4828 rad
  Diferencia: 0.000111 rad

  ✓ Coincide! El desfase medido corresponde al generado.


## 6. Test Inverso: Recuperar Flujo desde Desfase Generado

In [ ]:
print("\n" + "=" * 80)
print("TEST INVERSO: Recuperar Flujo desde Desfase")
print("=" * 80)

print("\n🔄 Ecuación de generación:")
print(f"   Δφ = √((ω×Δz²)/(4α)) + (v×Cw×Δz)/(2λ)")

print("\n🔄 Ecuación de recuperación (despejando v):")
print(f"   v = [Δφ - √((ω×Δz²)/(4α))] × (2λ)/(Cw×Δz)")

# Aplicar
v_recuperado = (delta_phi - term_conductive) * (2 * lambda_val) / (Cw * delta_z)
v_recuperado_mm_day = v_recuperado * 86400 * 1000

print(f"\nResultados:")
print(f"  v recuperado = {v_recuperado:.6e} m/s")
print(f"  v recuperado = {v_recuperado_mm_day:.4f} mm/día")
print(f"\n  v objetivo   = {v_objetivo:.6e} m/s")
print(f"  v objetivo   = {TARGET_FLUX:.4f} mm/día")

error_rel = abs(v_recuperado - v_objetivo) / v_objetivo * 100
print(f"\n  Error relativo: {error_rel:.2f}%")

if error_rel < 1:
    print("\n   ÉXITO: La ecuación inversa funciona correctamente!")
else:
    print(f"\n  ERROR: Hay inconsistencia entre generación y recuperación")


TEST INVERSO: Recuperar Flujo desde Desfase

🔄 Ecuación de generación:
   Δφ = √((ω×Δz²)/(4α)) + (v×Cw×Δz)/(2λ)

🔄 Ecuación de recuperación (despejando v):
   v = [Δφ - √((ω×Δz²)/(4α))] × (2λ)/(Cw×Δz)

Resultados:
  v recuperado = 5.892919e-08 m/s
  v recuperado = 5.0915 mm/día

  v objetivo   = 5.787037e-08 m/s
  v objetivo   = 5.0000 mm/día

  Error relativo: 1.83%

  ❌ ERROR: Hay inconsistencia entre generación y recuperación


## 7. Comparar con Ecuación Actual en vflux_methods.py

In [ ]:
print("\n" + "=" * 80)
print("COMPARACIÓN: Ecuación Actual vs Ecuación Correcta")
print("=" * 80)

# Ecuación ACTUAL en vflux_methods.py
v_actual = (4 * alpha * delta_phi) / (omega * delta_z**2)
v_actual_mm_day = v_actual * 86400 * 1000

print("\n Ecuación ACTUAL en vflux_methods.py:")
print("   v = (4 × α × Δφ) / (ω × Δz²)")
print(f"   v = {v_actual:.6e} m/s = {v_actual_mm_day:.2f} mm/día")

# Ecuación CORRECTA
print("\n Ecuación CORRECTA (con término conductivo):")
print("   v = [Δφ - √((ω×Δz²)/(4α))] × (2λ)/(Cw×Δz)")
print(f"   v = {v_recuperado:.6e} m/s = {v_recuperado_mm_day:.4f} mm/día")

print("\n" + "=" * 80)
print("RESUMEN")
print("=" * 80)
print(f"\n  Flujo objetivo:     {TARGET_FLUX:>10.2f} mm/día")
print(f"  Ecuación actual:    {v_actual_mm_day:>10.2f} mm/día  (error: {abs(v_actual_mm_day - TARGET_FLUX)/TARGET_FLUX*100:.0f}%)")
print(f"  Ecuación correcta:  {v_recuperado_mm_day:>10.2f} mm/día  (error: {abs(v_recuperado_mm_day - TARGET_FLUX)/TARGET_FLUX*100:.1f}%)")

print("\n" + "=" * 80)
print(" CONCLUSIÓN: La ecuación actual NO incluye el término conductivo")
print("   y por eso produce valores órdenes de magnitud incorrectos.")
print("=" * 80)


COMPARACIÓN: Ecuación Actual vs Ecuación Correcta

📌 Ecuación ACTUAL en vflux_methods.py:
   v = (4 × α × Δφ) / (ω × Δz²)
   v = 2.125117e+00 m/s = 183610101.79 mm/día

📌 Ecuación CORRECTA (con término conductivo):
   v = [Δφ - √((ω×Δz²)/(4α))] × (2λ)/(Cw×Δz)
   v = 5.892919e-08 m/s = 5.0915 mm/día

RESUMEN

  Flujo objetivo:           5.00 mm/día
  Ecuación actual:    183610101.79 mm/día  (error: 3672201936%)
  Ecuación correcta:        5.09 mm/día  (error: 1.8%)

🔍 CONCLUSIÓN: La ecuación actual NO incluye el término conductivo
   y por eso produce valores órdenes de magnitud incorrectos.
